<a href="https://colab.research.google.com/github/dlnguyenvlu/hackerrank/blob/main/3D_crystal_structure_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install m3gnet pyxtal ase
!cp /content/drive/MyDrive/Matsci/_dynamics.py /usr/local/lib/python3.8/dist-packages/m3gnet/models/_dynamics.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf  /content/MgB3C3
!mkdir  /content/MgB3C3
%cd /content/MgB3C3
#code to generate 100 2D crystal structure from given composition
#2D crystal only have spacegroup from 1-80
# This part is to test which spacegroup compatible with the given composition
from pyxtal import pyxtal
allow_sp=[]
for i in range(1,231):
    try:
        c = pyxtal()
        c.from_random(3,i, ['Mg','B','C'], [2,8,4])
        #ase_struc = c.to_ase()
        #ase_struc.write('POSCAR-'+str(i), format='vasp', vasp5=True, direct=True)
        allow_sp.append(i)
    except Exception as e:
        pass
print('spgnum = ',' '.join(str(v) for v in allow_sp))
#Now generate 3D crystal with compatible spacegroup
#import random
#for i in range(1,301):
#        c = pyxtal()
#        c.from_random(3,random.choice(allow_sp), ['Mg','B','C'], [1,3,3], thickness=2.5)
#        ase_struc = c.to_ase()
#        ase_struc.write('POSCAR-'+str("{:03d}".format(i)), format='vasp', vasp5=True, direct=True)
#        print('Done '+str(i))

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/content/MgB3C3
spgnum =  1 2 3 4 5 6 7 8 10 11 12 13 14 16 17 18 21 23 25 26 27 28 30 31 32 34 35 38 44 47 48 49 50 51 53 55 58 59 65 71 75 77 79 81 82 83 84 85 86 87 89 90 93 94 97 99 100 101 102 103 104 105 107 111 112 113 114 115 116 117 118 119 121 123 124 125 126 127 128 129 131 132 134 136 137 139 143 147 149 150 156 157 158 159 162 163 164 165 168 173 174 175 176 177 182 183 184 185 186 187 188 189 190 191 192 193 194 195 201 208 215 224


In [ ]:
import pandas as pd
import subprocess
import warnings,os    
from m3gnet.models import Relaxer
from m3gnet.models import M3GNet, M3GNetCalculator, Potential    
from pymatgen.core import Lattice, Structure
from ase.io import read
from ase.io.vasp import read_vasp
from spglib import get_symmetry,get_spacegroup    
for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")    
from pymatgen.io.vasp import Poscar
from ase.units import GPa

matlist = []
for i in range(1,301):
    structure = Structure.from_file("/content/MgB3C3/POSCAR-"+str("{:03d}".format(i)))
    df = pd.read_csv("/content/drive/MyDrive/Matsci/bulk_database.csv",names=['element','energy'], header=None)
    poscar = Poscar.from_file("/content/MgB3C3/POSCAR-"+str("{:03d}".format(i)))
    struct = poscar.structure
    poscar_ase = read("/content/MgB3C3/POSCAR-"+str("{:03d}".format(i)))
    #print('Doing .. ',struct.composition.reduced_formula)
    #print("Spacegroup:",get_spacegroup(poscar_ase))        
    relaxer = Relaxer(stress_weight=0.00624, optimizer="BFGS",constraints_sym=True)
    try:
        relax_results = relaxer.relax(struct, verbose=True,fmax=0.01)
        final_structure = relax_results['final_structure']
        final_energy = float(relax_results['trajectory'].energies[-1])
        final_energy_per_atom = float(relax_results['trajectory'].energies[-1] / len(struct))    
        #print(f"Relaxed lattice parameter is {final_structure.lattice.abc[0]:.3f} Å")
        relax_results['final_structure'].to("/content/MgB3C3/CONTCAR-"+str("{:03d}".format(i)),'poscar')
        cell = read_vasp("/content/MgB3C3/CONTCAR-"+str("{:03d}".format(i)))
        matlist.append(
            {
                'material_id': str("POSCAR-"+"{:03d}".format(i)),
                'spacegroup': get_spacegroup(cell),
                'Energy':  final_energy
            }
                      )
    except RuntimeError as e:
        pass

df = pd.DataFrame(matlist)
df.to_csv('/content/drive/MyDrive/random_search.csv')

      Step     Time          Energy         fmax
BFGS:    0 02:00:52      -15.819592       95.5713
BFGS:    1 02:00:52      -26.777349       14.5878
BFGS:    2 02:00:53      -30.708191       12.4490


/usr/local/lib/python3.8/dist-packages/ase/spacegroup/symmetrize.py:217: UserWarning: FixSymmetry adjust_cell may be ill behaved with large deformation gradient step 0.22140275816017008
  warnings.warn('FixSymmetry adjust_cell may be ill behaved with'
/usr/local/lib/python3.8/dist-packages/ase/spacegroup/symmetrize.py:217: UserWarning: FixSymmetry adjust_cell may be ill behaved with large deformation gradient step 0.1812692469220183
  warnings.warn('FixSymmetry adjust_cell may be ill behaved with'


BFGS:    3 02:00:59      -34.368362       17.9325


/usr/local/lib/python3.8/dist-packages/ase/spacegroup/symmetrize.py:217: UserWarning: FixSymmetry adjust_cell may be ill behaved with large deformation gradient step 0.18126924692201818
  warnings.warn('FixSymmetry adjust_cell may be ill behaved with'


BFGS:    4 02:01:06      -37.170372       11.7471
BFGS:    5 02:01:06      -30.870300       46.1129


...

BFGS:    7 03:47:25      -42.814171        0.7438
BFGS:    8 03:47:25      -42.828960        0.6156
BFGS:    9 03:47:25      -42.832115        0.1266
BFGS:   10 03:47:25      -42.832279        0.0300
BFGS:   11 03:47:25      -42.832294        0.0326
BFGS:   12 03:47:25      -42.832302        0.0114
BFGS:   13 03:47:26      -42.832306        0.0017
      Step     Time          Energy         fmax
BFGS:    0 03:47:33       14.629734      256.2408
BFGS:    1 03:47:33      -30.666906       31.2539
BFGS:    2 03:47:40      -32.570099       21.3928
BFGS:    3 03:47:46      -36.198177        9.8764
BFGS:    4 03:47:47      -34.758362       28.1224
BFGS:    5 03:47:47      -37.155983        4.6699


/usr/local/lib/python3.8/dist-packages/ase/spacegroup/symmetrize.py:217: UserWarning: FixSymmetry adjust_cell may be ill behaved with large deformation gradient step 0.16431679188127601
  warnings.warn('FixSymmetry adjust_cell may be ill behaved with'


BFGS:    6 03:47:47      -37.533367        2.7762
BFGS:    7 03:47:47      -38.058289        5.5683
BFGS:    8 03:47:47      -38.308182        4.8551
BFGS:    9 03:47:47      -38.550880        2.4919
BFGS:   10 03:47:47      -38.593227        3.1504
BFGS:   11 03:47:48      -38.627720        2.6506
BFGS:   12 03:47:48      -38.665176        0.4457
BFGS:   13 03:47:48      -38.666672        0.1687
BFGS:   14 03:47:48      -38.666927        0.0222
BFGS:   15 03:47:48      -38.666935        0.0043
      Step     Time          Energy         fmax
BFGS:    0 03:47:59      -23.509190       30.4861


/usr/local/lib/python3.8/dist-packages/ase/spacegroup/symmetrize.py:217: UserWarning: FixSymmetry adjust_cell may be ill behaved with large deformation gradient step 0.21368394510887723
  warnings.warn('FixSymmetry adjust_cell may be ill behaved with'


BFGS:    1 03:48:07      -30.741905       28.3448
BFGS:    2 03:48:07      -37.101803       18.1305
BFGS:    3 03:48:07      -42.856117        8.3270
BFGS:    4 03:48:07      -44.265350       15.3698
BFGS:    5 03:48:07      -45.173447        2.1250
BFGS:    6 03:48:07      -45.304966        1.0210
BFGS:    7 03:48:07      -45.345081        0.5152
BFGS:    8 03:48:07      -45.358467        0.4143
BFGS:    9 03:48:07      -45.371700        0.5870
BFGS:   10 03:48:08      -45.382908        0.6637
BFGS:   11 03:48:08      -45.392586        0.6075
BFGS:   12 03:48:08      -45.400200        0.4473
BFGS:   13 03:48:08      -45.404793        0.3339
BFGS:   14 03:48:08      -45.410496        0.4182
BFGS:   15 03:48:08      -45.406235        1.5723
BFGS:   16 03:48:08      -45.421574        0.5069
BFGS:   17 03:48:08      -45.431015        0.6087
BFGS:   18 03:48:09      -45.187904        3.2628
BFGS:   19 03:48:09      -45.439732        0.7166
BFGS:   20 03:48:09      -45.446365        0.7901
